In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, losses, metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

In [ ]:
!unzip drive/MyDrive/Colab\ Notebooks/ROBOTICS/project-jetbot/put_jetbot_dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/1652876013.741493/0632.jpg  
  inflating: dataset/1652876013.741493/0661.jpg  
  inflating: dataset/1652876013.741493/0622.jpg  
  inflating: dataset/1652876013.741493/0375.jpg  
  inflating: dataset/1652876013.741493/0302.jpg  
  inflating: dataset/1652876013.741493/0227.jpg  
  inflating: dataset/1652876013.741493/0324.jpg  
  inflating: dataset/1652876013.741493/0448.jpg  
  inflating: dataset/1652876013.741493/0383.jpg  
  inflating: dataset/1652876013.741493/0402.jpg  
  inflating: dataset/1652876013.741493/0146.jpg  
  inflating: dataset/1652876013.741493/0228.jpg  
  inflating: dataset/1652876013.741493/0378.jpg  
  inflating: dataset/1652876013.741493/0169.jpg  
  inflating: dataset/1652876013.741493/0645.jpg  
  inflating: dataset/1652876013.741493/0120.jpg  
  inflating: dataset/1652876013.741493/0366.jpg  
  inflating: dataset/1652876013.741493/0541.jpg  
  inflating: dataset/1652876013.741493/0415.jpg  

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
import os

X = []
y = []

DS = Path("dataset")

recordings = sorted(DS.glob('*.csv'))
for recording in recordings:
    csvfile = Path(recording).name
    df = pd.read_csv(DS / csvfile, header=None)
    y.append(df[[1, 2]].to_numpy())
    recordid = csvfile[:-4]
    img_files = sorted((DS / recordid).glob('*.jpg'))
    for filename in img_files:
        file = cv2.imread(str(filename))
        X.append(file / 255.0)

X = np.array(X)
y = np.concatenate(y)

ValueError: need at least one array to concatenate

In [ ]:
y.shape

(7584, 2)

In [ ]:
X.shape

(7584, 224, 224, 3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(24, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='relu'))

Network shouldn't be too big (maybe around 600k)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 24)      3480      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 24)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        6944      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 32)        0

In [ ]:
model.compile(optimizer='adam',
              loss=losses.MeanSquaredError(),
              metrics=[metrics.MeanAbsoluteError(),
                       metrics.RootMeanSquaredError()])

checkpoint_cb = ModelCheckpoint('model_checkpoint.h5', save_best_only=True)
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(factor=0.5, patience=5, min_lr=0.00001)
tensorboard_cb = TensorBoard(log_dir='./logs', histogram_freq=1)

history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb, tensorboard_cb])

Epoch 1/100
190/190 [==============================] - ETA: 0s - loss: 0.6216 - mean_absolute_error: 0.6538 - root_mean_squared_error: 0.7884

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


190/190 [==============================] - 10s 23ms/step - loss: 0.6216 - mean_absolute_error: 0.6538 - root_mean_squared_error: 0.7884 - val_loss: 0.6112 - val_mean_absolute_error: 0.6547 - val_root_mean_squared_error: 0.7818 - lr: 0.0010
Epoch 2/100
190/190 [==============================] - 3s 16ms/step - loss: 0.6095 - mean_absolute_error: 0.6524 - root_mean_squared_error: 0.7807 - val_loss: 0.6112 - val_mean_absolute_error: 0.6547 - val_root_mean_squared_error: 0.7818 - lr: 0.0010
Epoch 3/100
190/190 [==============================] - 3s 16ms/step - loss: 0.6095 - mean_absolute_error: 0.6524 - root_mean_squared_error: 0.7807 - val_loss: 0.6112 - val_mean_absolute_error: 0.6547 - val_root_mean_squared_error: 0.7818 - lr: 0.0010
Epoch 4/100
190/190 [==============================] - 3s 16ms/step - loss: 0.6095 - mean_absolute_error: 0.6524 - root_mean_squared_error: 0.7807 - val_loss: 0.6112 - val_mean_absolute_error: 0.6547 - val_root_mean_squared_error: 0.7818 - lr: 0.0010
Epoch 5

In [ ]:
!pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.4 MB/s eta 0:00:00


In [ ]:
import tf2onnx
import onnx

onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)
onnx.save(onnx_model, "drive/MyDrive/model.onnx")

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path="/gdrive/My Drive/model.onnx")